<a href="https://colab.research.google.com/github/MNderi/verityVault/blob/main/SquenceTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
pip install accelerate -U


In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# Import necessary libraries
import argparse
from torch import nn
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Features, Value, ClassLabel, load_metric
import numpy as np
import os
import json
from sklearn.metrics import classification_report, confusion_matrix
import glob, shutil

# Function to load and tokenize the data
def load_data(path, class_list, tokenizer_name="", num_classes=13):
    features = Features({'text': Value('string'), 'label': ClassLabel(num_classes, class_list)})
    dataset = load_dataset('csv', data_files={"test": path},
                        delimiter=',', column_names=['text', 'label'],
                        skiprows=1, features=features,
                        keep_in_memory=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512 if "nli-bert" in tokenizer_name else None)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    test_dataset = tokenized_datasets["test"]

    return test_dataset

# Main function to train the model
def main(model_name, directory, tokenizer_name, dataset_train, dataset_val, dataset_test, num_classes, class_list=None, problem_type=None, epochs=10, run_num=1):

    features = Features({'text': Value('string'), 'label': ClassLabel(num_classes, class_list)})
    dataset = load_dataset('csv', data_files={"train": dataset_train,
                                            "val": dataset_val,
                                            "test": dataset_test},
                        delimiter=',', column_names=['text', 'label'],
                        skiprows=1, features=features,
                        keep_in_memory=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512 if "nli-bert" in tokenizer_name else None)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    train_dataset = tokenized_datasets["train"].shuffle(seed=42)
    eval_dataset = tokenized_datasets["val"]
    test_dataset = tokenized_datasets["test"]

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True, problem_type=problem_type)

    metric = load_metric("accuracy")
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
        prec = precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
        rec = recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
        f1w = f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
        f1micro = f1.compute(predictions=predictions, references=labels, average="micro")["f1"]
        return {"accuracy": acc, "precision": prec, "recall": rec, "f1_weighted": f1w, "f1_micro": f1micro}

    training_args = TrainingArguments(output_dir=directory,
                                    do_train=True,
                                    do_eval=True,
                                    evaluation_strategy="epoch",
                                    per_device_train_batch_size=32 if "deberta" not in tokenizer_name and "electra" not in tokenizer_name else 16,
                                    per_device_eval_batch_size=32 if "deberta" not in tokenizer_name and "electra" not in tokenizer_name else 16,
                                    learning_rate=5e-5,
                                    logging_strategy="epoch",
                                    num_train_epochs=epochs,
                                    save_strategy="epoch",
                                    save_total_limit=1,
                                    load_best_model_at_end=True,
                                    metric_for_best_model="eval_f1_weighted",
                                    lr_scheduler_type="cosine")

    class CustomTrainer(Trainer):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    checkpoints = glob.glob(directory + "/checkpoint*")
    for c in checkpoints:
        shutil.rmtree(c)

    trainer.train()
    with open(os.path.join(directory, f"{tokenizer_name.replace('/', '_') + str(run_num)}.log"), "w+") as f:
        f.write(json.dumps(trainer.predict(test_dataset).metrics))

# Argument parsing
model_name = "bert-base-uncased"
directory = "./big_bench_model"
tokenizer_name = "bert-base-uncased"
dataset_train = "/content/drive/MyDrive/Trainer/GeneralClassification/train.csv"
dataset_val = "/content/drive/MyDrive/Trainer/GeneralClassification/dev.csv"
dataset_test = "/content/drive/MyDrive/Trainer/GeneralClassification/test.csv"
num_classes = 2
class_list = ["negative", "positive"]
epochs = 5

main(model_name, directory, tokenizer_name,
    dataset_train,
    dataset_val,
    dataset_test,
    num_classes=num_classes,
    class_list=class_list,
    problem_type="single_label_classification",
    epochs=epochs)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-1-62442ea98f02>:55: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Weighted,F1 Micro
1,0.601900,0.430838,0.780952,0.782141,0.780952,0.779745,0.780952
2,0.365200,0.299087,0.823810,0.866758,0.823810,0.820843,0.823810
3,0.294200,0.297610,0.826190,0.868087,0.826190,0.823376,0.826190
4,0.245400,0.295385,0.823810,0.843939,0.823810,0.822848,0.823810
5,0.217700,0.296495,0.819048,0.823094,0.819048,0.819212,0.819048


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, Features, Value, ClassLabel, load_metric
import numpy as np
import os
import json
import shutil
import glob

# Function to load and tokenize the data
def load_data(path, class_list, tokenizer_name="", num_classes=2):
    features = Features({'text': Value('string'), 'label': ClassLabel(num_classes, class_list)})
    dataset = load_dataset('csv', data_files={"test": path},
                        delimiter=',', column_names=['text', 'label'],
                        skiprows=1, features=features,
                        keep_in_memory=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512 if "nli-bert" in tokenizer_name else None)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    test_dataset = tokenized_datasets["test"]

    return test_dataset

# Main function to train the model
def main(model_name, directory, tokenizer_name, dataset_train, dataset_val, dataset_test, num_classes, class_list=None, problem_type=None, epochs=5):

    features = Features({'text': Value('string'), 'label': ClassLabel(num_classes, class_list)})
    dataset = load_dataset('csv', data_files={"train": dataset_train,
                                            "val": dataset_val,
                                            "test": dataset_test},
                        delimiter=',', column_names=['text', 'label'],
                        skiprows=1, features=features,
                        keep_in_memory=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512 if "nli-bert" in tokenizer_name else None)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    train_dataset = tokenized_datasets["train"].shuffle(seed=42)
    eval_dataset = tokenized_datasets["val"]
    test_dataset = tokenized_datasets["test"]

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, problem_type=problem_type)

    metric = load_metric("accuracy")
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
        prec = precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
        rec = recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
        f1w = f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
        f1micro = f1.compute(predictions=predictions, references=labels, average="micro")["f1"]
        return {"accuracy": acc, "precision": prec, "recall": rec, "f1_weighted": f1w, "f1_micro": f1micro}

    training_args = TrainingArguments(output_dir=directory,
                                    do_train=True,
                                    do_eval=True,
                                    evaluation_strategy="epoch",
                                    per_device_train_batch_size=32 if "deberta" not in tokenizer_name and "electra" not in tokenizer_name else 16,
                                    per_device_eval_batch_size=32 if "deberta" not in tokenizer_name and "electra" not in tokenizer_name else 16,
                                    learning_rate=5e-5,
                                    logging_strategy="epoch",
                                    num_train_epochs=epochs,
                                    save_strategy="epoch",
                                    save_total_limit=1,
                                    load_best_model_at_end=True,
                                    metric_for_best_model="eval_f1_weighted",
                                    lr_scheduler_type="cosine")

    class CustomTrainer(Trainer):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Save the model after the fourth epoch
    if trainer.state.global_step == 4 * len(train_dataset) // training_args.per_device_train_batch_size:
        trainer.save_model(os.path.join(directory, "/content/drive/MyDrive/Trainer/GeneralClassification/broadmodel"))

    with open(os.path.join(directory, f"{tokenizer_name.replace('/', '_')}.log"), "w+") as f:
        f.write(json.dumps(trainer.predict(test_dataset).metrics))

# Argument parsing
model_name = "bert-base-uncased"
directory = "./big_bench_model"
tokenizer_name = "bert-base-uncased"
dataset_train = "/content/drive/MyDrive/Trainer/GeneralClassification/train.csv"
dataset_val = "/content/drive/MyDrive/Trainer/GeneralClassification/dev.csv"
dataset_test = "/content/drive/MyDrive/Trainer/GeneralClassification/test.csv"
num_classes = 2
class_list = ["negative", "positive"]
epochs = 5

main(model_name, directory, tokenizer_name,
    dataset_train,
    dataset_val,
    dataset_test,
    num_classes=num_classes,
    class_list=class_list,
    problem_type="single_label_classification",
    epochs=epochs)


Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Weighted,F1 Micro
1,0.664100,0.570266,0.673810,0.740694,0.673810,0.658339,0.673810
2,0.449800,0.338645,0.804762,0.850963,0.804762,0.800949,0.804762
3,0.294900,0.310576,0.823810,0.869377,0.823810,0.820610,0.823810
4,0.236100,0.323382,0.823810,0.864239,0.823810,0.821068,0.823810
5,0.214400,0.310939,0.826190,0.865601,0.826190,0.823593,0.826190


In [ ]:
from transformers import AutoModelForSequenceClassification

# Assuming your model is already loaded
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Define path to save the model
save_path = "/content/drive/MyDrive/Trainer/GeneralClassification/model"

# Save the model
model.save_pretrained(save_path)

# Optionally, save the tokenizer too if needed
# tokenizer.save_pretrained(save_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install onnxruntime-tools



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00


In [ ]:
from transformers import convert_graph_to_onnx
from pathlib import Path

# Specify tokenizer and output path
tokenizer_name = "bert-base-uncased"
output_path = Path("/content/drive/MyDrive/Trainer/GeneralClassification/modelonx/model.onnx")

# Convert model to ONNX format, use opset 14
convert_graph_to_onnx.convert(framework="pt", model=save_path, output=output_path, opset=14, tokenizer=tokenizer_name) # Change opset to 14

/usr/local/lib/python3.10/dist-packages/transformers/convert_graph_to_onnx.py:361: FutureWarning: The `transformers.convert_graph_to_onnx` package is deprecated and will be removed in version 5 of Transformers
  warnings.warn(


ONNX opset version set to: 14
Loading pipeline (model: /content/drive/MyDrive/Trainer/GeneralClassification/model, tokenizer: bert-base-uncased)
Using framework PyTorch: 2.3.0+cu121
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


In [ ]:
from torch import nn
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Features, Value, ClassLabel, load_metric
import numpy as np
import os
import json
from sklearn.metrics import classification_report, confusion_matrix
import glob
import shutil

def load_data(path, class_list, tokenizer_name="", num_classes=13):
    features = Features({'text': Value('string'), 'label': ClassLabel(num_classes, class_list)})
    dataset = load_dataset('csv', data_files={"test": path},
                           delimiter=',', column_names=['text', 'label'],
                           skiprows=1, features=features,
                           keep_in_memory=True)

    # Load tokenizer and tokenize
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512 if "nli-bert" in tokenizer_name else None)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    test_dataset = tokenized_datasets["test"]

    return test_dataset

def main(model_name, directory, tokenizer_name, num_classes, class_list=None, use_label_encoder=True, problem_type=None, epochs=10, weights=None, run_num=1):

    # Define datasets paths directly here
    big_bench_train_dataset = "/content/drive/MyDrive/Trainer/GeneralClassification/train.csv"
    big_bench_val_dataset = "/content/drive/MyDrive/Trainer/GeneralClassification/dev.csv"
    big_bench_test_dataset = "/content/drive/MyDrive/Trainer/GeneralClassification/test.csv"
    coarse_train_dataset = "/content/drive/MyDrive/Trainer/CoarseClassData/train_dataset.csv"
    coarse_val_dataset = "/content/drive/MyDrive/Trainer/CoarseClassData/val_dataset.csv"
    coarse_test_dataset = "/content/drive/MyDrive/Trainer/CoarseClassData/test_dataset.csv"
    fine_train_dataset = "/content/drive/MyDrive/Trainer/FineClassData/train_dataset.csv"
    fine_val_dataset = "/content/drive/MyDrive/Trainer/FineClassData/val_dataset.csv"
    fine_test_dataset = "/content/drive/MyDrive/Trainer/FineClassData/test_dataset.csv"

    features = Features({'text': Value('string'), 'label': ClassLabel(num_classes, class_list)})
    dataset = load_dataset('csv', data_files={"train": big_bench_train_dataset,
                                              "val": big_bench_val_dataset,
                                              "test": big_bench_test_dataset},
                           delimiter=',', column_names=['text', 'label'],
                           skiprows=1, features=features,
                           keep_in_memory=True)

    # Load tokenizer and tokenize
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512 if "nli-bert" in tokenizer_name else None)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Extract splits
    train_dataset = tokenized_datasets["train"].shuffle(seed=42)
    eval_dataset = tokenized_datasets["val"]
    test_dataset = tokenized_datasets["test"]

    # Load model with new head
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True, problem_type=problem_type)

    # Define metrics
    metric = load_metric("accuracy")
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
        prec = precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
        rec = recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
        f1w = f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
        f1micro = f1.compute(predictions=predictions, references=labels, average="micro")["f1"]
        return {"accuracy": acc, "precision": prec, "recall": rec, "f1_weighted": f1w, "f1_micro": f1micro}

    # Define training arguments for trainer
    training_args = TrainingArguments(output_dir=directory,
                                      do_train=True,
                                      do_eval=True,
                                      evaluation_strategy="epoch",
                                      per_device_train_batch_size=32 if "deberta" not in tokenizer_name and "electra" not in tokenizer_name else 16,
                                      per_device_eval_batch_size=32 if "deberta" not in tokenizer_name and "electra" not in tokenizer_name else 16,
                                      learning_rate=5e-5,
                                      logging_strategy="epoch",
                                      num_train_epochs=epochs,
                                      save_strategy="epoch",
                                      save_total_limit=1,
                                      load_best_model_at_end=True,
                                      metric_for_best_model="eval_f1_weighted",
                                      lr_scheduler_type="cosine")

    class CustomTrainer(Trainer):
        def __init__(self, weights, **kwargs):
            super().__init__(**kwargs)
            self.weights = weights

        def compute_loss(self, model, inputs, return_outputs=False):
            labels = inputs.get("labels")
            # forward pass
            outputs = model(**inputs)
            logits = outputs.get('logits')
            # compute custom loss
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(self.weights, device="cuda") if self.weights else None)
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            return (loss, outputs) if return_outputs else loss

    # Define trainer
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        weights=weights
    )

    checkpoints = glob.glob(directory + "/checkpoint*")
    for c in checkpoints:
        shutil.rmtree(c)

    # Train
    trainer.train()
    with open(os.path.join(directory, f"{tokenizer_name.replace('/', '_') + str(run_num)}.log"), "w+") as f:
        f.write(json.dumps(trainer.predict(test_dataset).metrics))

    if num_classes == 13:
        class_list = ['faulty generalization', 'false causality', 'circular reasoning', 'ad populum', 'ad hominem', 'fallacy of logic', 'appeal to emotion', 'false dilemma', 'equivocation', 'fallacy of extension', 'fallacy of relevance', 'fallacy of credibility', 'intentional']
        test_dataset = load_data("/content/drive/MyDrive/Trainer/FineClassData/test_dataset.csv", class_list, tokenizer_name=tokenizer_name, num_classes=len(class_list))
        y_pred = torch.tensor(trainer.predict(test_dataset).predictions, dtype=torch.float32).max(1).indices.numpy()
        print("Classification report:\n\n", classification_report(test_dataset["label"], y_pred, target_names=class_list, digits=4))
        print(confusion_matrix(test_dataset["label"], y_pred, labels=list(range(0, len(class_list), 1))))


if __name__ == "__main__":
    main(
        model_name="bert-base-uncased",
        directory="output_directory",
        tokenizer_name="bert-base-uncased",
        num_classes=2,
        class_list=["negative", "positive"],
        problem_type="single_label_classification",
        epochs=5,
        weights=None,
        run_num=1
    )


Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Weighted,F1 Micro
1,0.653600,0.486203,0.728571,0.815812,0.728571,0.714776,0.728571
2,0.397200,0.310918,0.828571,0.828909,0.828571,0.828665,0.828571
3,0.280000,0.288554,0.838095,0.854062,0.838095,0.837555,0.838095
4,0.236700,0.290093,0.830952,0.855732,0.830952,0.829678,0.830952
5,0.209400,0.289764,0.816667,0.830048,0.816667,0.816220,0.816667


In [ ]:
from transformers import AutoModelForSequenceClassification

# Assuming your model is already loaded
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Define path to save the model
save_path = "/content/drive/MyDrive/Trainer/GeneralClassification/model"

# Save the model
model.save_pretrained(save_path)

# Optionally, save the tokenizer too if needed
# tokenizer.save_pretrained(save_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
